In [49]:
import re
import os
import sys

sys.path.append("/home/admin01/Data/BBSEA-sapien/")

In [41]:
from config.core import dataset_path, prompts_dir
from llm_interaction.request import gpt_api

In [78]:
def extract_pre_scene_graph(lines):
    pre_scene_graph_lines = []
    inside_pre_scene_graph = False

    for line in lines:
        if "pre scenegraph:" in line:
            inside_pre_scene_graph = True
            continue
        if "post scenegraph:" in line:
            break
        if inside_pre_scene_graph:
            pre_scene_graph_lines.append(line)

    return ''.join(pre_scene_graph_lines)


task_list = os.listdir(dataset_path)
task_list.remove("result.json")
task_list.sort(key=lambda x : int(x.split('_')[1]))

task_decomposition_records=[]

for task in task_list:
    task_decomposition_record={}

    with open(os.path.join(dataset_path, task, "description.txt"), 'r') as f:
        task_log = f.readlines()
        task_decomposition_record["task_description"]=task_log[1].strip()
        task_decomposition_record["task_pre_SG"]=extract_pre_scene_graph(task_log)

    subtask_list = os.listdir(os.path.join(dataset_path, task))
    subtask_list.remove("description.txt")
    subtask_list.sort(key=lambda x : int(x.split('_')[1]))
    
    subtask_descriptions=[]
    for subtask in subtask_list:
        with open(os.path.join(dataset_path, task, subtask, "description.txt"), 'r') as f:
            subtask_log = f.readlines()
            subtask_description = subtask_log[1].strip()
            subtask_descriptions.append(subtask_description)

    task_decomposition_record["subtask_descriptions"]=subtask_descriptions

    task_decomposition_records.append(task_decomposition_record)


with open(os.path.join(prompts_dir, "task_decomposition_quality_w_SG.txt"), "r") as f:
    base_prompt = f.read()

Scores=[]
multi_task_decomposition_prompt=""
for task_decomposition_record in task_decomposition_records:
    single_task_decomposition_prompt=\
    "Scene Description:\n{}\n\nMajor Task:\n{}\n\nTask Decomposition:\n{}\n\nPlease provide a score (1-5) based on the task decomposition above, considering the scene description, and briefly explain your reasoning.\n"

    multi_task_decomposition_prompt+=single_task_decomposition_prompt.format(
        task_decomposition_record["task_pre_SG"], 
        task_decomposition_record["task_description"], 
        task_decomposition_record["subtask_descriptions"]
    )

task_decomposition_quality_prompt = base_prompt.format(multi_task_decomposition_prompt)

messages = [
    {
        'role': 'user',
        'content': [
            {"type":"text", "text": task_decomposition_quality_prompt},
        ]
    },
]
response = gpt_api(messages, 0.2, model='4o')

scores = re.findall(r'Score: (\d)/5', response)
scores_as_floats = [float(score)/5 for score in scores]

print(scores_as_floats)

[0.8, 1.0, 0.6]


In [72]:

print(len(response))

1937


In [81]:
a=[0.6, 1.0, 0.8, 0.8, 1.0, 0.4]
print(sum(a)/len(a))

a=[0.6, 1.0, 0.8, 0.8, 1.0, 0.6]
print(sum(a)/len(a))

0.7666666666666667
0.7999999999999999


In [76]:
a=[0.6, 0.8, 0.8, 0.6, 0.8, 0.4]
print(sum(a)/len(a))

a=[0.6, 0.8, 1.0, 0.6, 0.8, 0.6]
print(sum(a)/len(a))

0.6666666666666669
0.7333333333333333


In [42]:
messages = [
    {
        'role': 'user',
        'content': [
            {"type":"text", "text": task_decomposition_quality_prompt},
        ]
    },
]
response = gpt_api(messages, 0.2, model='4o')

In [44]:
print(response)

```
Score: 4/5
Reason:
- Logical Coherence: The task decomposition is logically coherent. It clearly separates the toys, kitchen items, and the power drill into distinct areas on the table.
- Detailing: The smaller tasks are clear and specific, detailing exactly which items need to be moved and to which part of the table.
- Goal Achievement: The tasks effectively achieve the major goal of organizing the items into their respective categories and areas on the table.
- Consideration of Spatial Relationships: The task considers spatial relationships well by grouping similar items together and placing the power drill in a separate area. However, it could be improved by specifying the exact positions within the right side or center of the table for better clarity.
```


In [48]:
import re

text = """
Score: 4/5
Reason:
- Logical Coherence: The task decomposition is logically coherent. It clearly separates the toys, kitchen items, and the power drill into distinct areas on the table.
- Detailing: The smaller tasks are clear and specific, detailing exactly which items need to be moved and to which part of the table.
- Goal Achievement: The tasks effectively achieve the major goal of organizing the items into their respective categories and areas on the table.
- Consideration of Spatial Relationships: The task considers spatial relationships well by grouping similar items together and placing the power drill in a separate area. However, it could be improved by specifying the exact positions within the right side or center of the table for better clarity.
"""

# Regular expression to extract score
match = re.search(r'Score: (\d+)/(\d+)', text)
if match:
    score = float(match.group(1)) / float(match.group(2))
    print(f"Score: {score}")
else:
    print("Score not found")


Score: 0.8
